# 02 — Feature Engineering
**Smart-Lite Insight: Building the Feature Pipeline**

This notebook develops and tests the feature engineering pipeline that transforms
raw energy readings into ML-ready features. Every feature created here will be
extracted into `src/features.py` as a reusable module.

Features we'll build:
1. **Temporal** — hour, day of week, month, weekend flag, time-of-use period
2. **Lag** — previous readings at 1, 5, 15, 60 minute offsets
3. **Rolling statistics** — mean, std, min, max over 1h, 6h, 24h windows
4. **Rate of change** — first derivative of power consumption
5. **Sub-metering ratios** — proportion of metered vs unmetered load
6. **Cyclical encodings** — sin/cos transforms for hour and day of week

In [ ]:
import sqlite3
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", 30)
pd.set_option("display.float_format", "{:.4f}".format)
plt.rcParams["figure.figsize"] = (14, 5)
plt.rcParams["figure.dpi"] = 100
plt.style.use("seaborn-v0_8-whitegrid")

DB_PATH = "../data/processed/energy.db"
print(f"Database: {Path(DB_PATH).resolve()}")

## 1. Load Raw Data

In [ ]:
conn = sqlite3.connect(DB_PATH)
df = pd.read_sql_query(
    """
    SELECT
        timestamp,
        global_active_power_kw,
        global_reactive_power_kw,
        voltage_v,
        global_intensity_a,
        sub_metering_1_wh,
        sub_metering_2_wh,
        sub_metering_3_wh
    FROM readings
    WHERE site_id = 'home-01'
    ORDER BY timestamp
    """,
    conn,
    parse_dates=["timestamp"],
)
conn.close()

df = df.set_index("timestamp")
print(f"Loaded {len(df):,} rows")
print(f"Columns: {list(df.columns)}")
df.head()

## 2. Temporal Features

Time-based features capture the cyclical nature of energy consumption.
People use more energy at certain hours, on certain days, and in certain seasons.

In [ ]:
def add_temporal_features(df: pd.DataFrame) -> pd.DataFrame:
    """Add time-based features from the timestamp index."""
    out = df.copy()

    # Basic extractions
    out["hour"] = out.index.hour
    out["day_of_week"] = out.index.dayofweek  # 0=Monday, 6=Sunday
    out["month"] = out.index.month
    out["is_weekend"] = (out.index.dayofweek >= 5).astype(int)

    # Time-of-use periods (common in energy tariffs)
    # Off-peak: 00-06, Mid-peak: 06-09 & 17-21, On-peak: 09-17, Evening: 21-00
    conditions = [
        (out["hour"] >= 0) & (out["hour"] < 6),
        (out["hour"] >= 6) & (out["hour"] < 9),
        (out["hour"] >= 9) & (out["hour"] < 17),
        (out["hour"] >= 17) & (out["hour"] < 21),
        (out["hour"] >= 21),
    ]
    choices = ["off_peak", "morning_peak", "daytime", "evening_peak", "late_evening"]
    out["time_of_use"] = np.select(conditions, choices, default="unknown")

    # Cyclical encodings (so hour 23 is close to hour 0)
    out["hour_sin"] = np.sin(2 * np.pi * out["hour"] / 24)
    out["hour_cos"] = np.cos(2 * np.pi * out["hour"] / 24)
    out["dow_sin"] = np.sin(2 * np.pi * out["day_of_week"] / 7)
    out["dow_cos"] = np.cos(2 * np.pi * out["day_of_week"] / 7)
    out["month_sin"] = np.sin(2 * np.pi * out["month"] / 12)
    out["month_cos"] = np.cos(2 * np.pi * out["month"] / 12)

    return out

df = add_temporal_features(df)
print(f"Shape after temporal features: {df.shape}")
df[["hour", "day_of_week", "is_weekend", "time_of_use",
    "hour_sin", "hour_cos"]].head(10)

In [ ]:
# Visualise cyclical encoding — hour_sin vs hour_cos should form a circle
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Hour cycle
hours = df.groupby("hour")[["hour_sin", "hour_cos"]].mean()
axes[0].scatter(hours["hour_sin"], hours["hour_cos"], c=hours.index, cmap="twilight", s=80, zorder=3)
for h in range(24):
    axes[0].annotate(str(h), (hours.loc[h, "hour_sin"], hours.loc[h, "hour_cos"]),
                     fontsize=8, ha="center", va="center")
axes[0].set_xlabel("hour_sin")
axes[0].set_ylabel("hour_cos")
axes[0].set_title("Cyclical Hour Encoding")
axes[0].set_aspect("equal")
axes[0].grid(True, alpha=0.3)

# Day of week cycle
dows = df.groupby("day_of_week")[["dow_sin", "dow_cos"]].mean()
day_names = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
axes[1].scatter(dows["dow_sin"], dows["dow_cos"], c=dows.index, cmap="Set2", s=80, zorder=3)
for d in range(7):
    axes[1].annotate(day_names[d], (dows.loc[d, "dow_sin"], dows.loc[d, "dow_cos"]),
                     fontsize=9, ha="center", va="center")
axes[1].set_xlabel("dow_sin")
axes[1].set_ylabel("dow_cos")
axes[1].set_title("Cyclical Day-of-Week Encoding")
axes[1].set_aspect("equal")
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Why Cyclical Encoding?

Standard integer encoding (hour=0,1,...,23) tells the model that hour 23 is
far from hour 0, when in reality they're adjacent. Sin/cos encoding preserves
this circular relationship. The same applies to day-of-week and month.

## 3. Lag Features

Lag features capture recent history — what was the power consumption
1, 5, 15, and 60 minutes ago? These are essential for time-series
anomaly detection because anomalies are defined relative to recent behaviour.

In [ ]:
def add_lag_features(df: pd.DataFrame, target_col: str = "global_active_power_kw",
                     lags: list[int] = None) -> pd.DataFrame:
    """Add lagged values of the target column.

    Args:
        df: DataFrame with time-series data.
        target_col: Column to create lags for.
        lags: List of lag offsets in minutes (default: [1, 5, 15, 60]).

    Returns:
        DataFrame with lag columns added.
    """
    if lags is None:
        lags = [1, 5, 15, 60]

    out = df.copy()
    for lag in lags:
        out[f"{target_col}_lag_{lag}m"] = out[target_col].shift(lag)

    return out

df = add_lag_features(df)
lag_cols = [c for c in df.columns if "_lag_" in c]
print(f"Lag features added: {lag_cols}")
df[["global_active_power_kw"] + lag_cols].head(65)

In [ ]:
# Visualise: current value vs lag_60m — should show strong correlation
sample = df.iloc[1000:2000]

fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(sample.index, sample["global_active_power_kw"], label="Current", alpha=0.8)
ax.plot(sample.index, sample["global_active_power_kw_lag_60m"], label="60min ago", alpha=0.6)
ax.set_xlabel("Time")
ax.set_ylabel("Power (kW)")
ax.set_title("Current vs 60-Minute Lag — Autocorrelation Visible")
ax.legend()
plt.tight_layout()
plt.show()

## 4. Rolling Statistics

Rolling windows capture the local trend and variability. A sudden spike
looks different depending on whether the rolling mean is high or low.

In [ ]:
def add_rolling_features(df: pd.DataFrame, target_col: str = "global_active_power_kw",
                         windows: list[int] = None) -> pd.DataFrame:
    """Add rolling mean, std, min, max for given window sizes.

    Args:
        df: DataFrame with time-series data.
        target_col: Column to compute rolling stats for.
        windows: Window sizes in minutes (default: [60, 360, 1440]).

    Returns:
        DataFrame with rolling statistic columns added.
    """
    if windows is None:
        windows = [60, 360, 1440]  # 1h, 6h, 24h

    out = df.copy()
    for w in windows:
        label = f"{w // 60}h" if w >= 60 else f"{w}m"
        rolling = out[target_col].rolling(window=w, min_periods=w // 2)

        out[f"{target_col}_roll_mean_{label}"] = rolling.mean()
        out[f"{target_col}_roll_std_{label}"] = rolling.std()
        out[f"{target_col}_roll_min_{label}"] = rolling.min()
        out[f"{target_col}_roll_max_{label}"] = rolling.max()

    return out

df = add_rolling_features(df)
roll_cols = [c for c in df.columns if "_roll_" in c]
print(f"Rolling features added ({len(roll_cols)}):")
for c in roll_cols:
    print(f"  {c}")

In [ ]:
# Visualise rolling features for a sample day
sample_start = df.index.min() + pd.Timedelta(days=35)
sample_end = sample_start + pd.Timedelta(days=3)
sample = df.loc[sample_start:sample_end]

fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Top: raw + rolling means
axes[0].plot(sample.index, sample["global_active_power_kw"],
             alpha=0.4, linewidth=0.5, label="Raw (1-min)", color="grey")
axes[0].plot(sample.index, sample["global_active_power_kw_roll_mean_1h"],
             label="1h mean", color="steelblue", linewidth=1.5)
axes[0].plot(sample.index, sample["global_active_power_kw_roll_mean_6h"],
             label="6h mean", color="coral", linewidth=1.5)
axes[0].plot(sample.index, sample["global_active_power_kw_roll_mean_24h"],
             label="24h mean", color="green", linewidth=1.5)
axes[0].set_ylabel("Power (kW)")
axes[0].set_title("Rolling Means at Different Window Sizes")
axes[0].legend(loc="upper right")

# Bottom: rolling std (volatility)
axes[1].plot(sample.index, sample["global_active_power_kw_roll_std_1h"],
             label="1h std", color="steelblue", linewidth=1.5)
axes[1].plot(sample.index, sample["global_active_power_kw_roll_std_6h"],
             label="6h std", color="coral", linewidth=1.5)
axes[1].set_ylabel("Std Dev (kW)")
axes[1].set_title("Rolling Volatility — High Std = Unstable Consumption")
axes[1].legend(loc="upper right")

plt.tight_layout()
plt.show()

## 5. Rate of Change

The first derivative of power consumption — how fast is usage changing?
Sudden large changes are often anomalous.

In [ ]:
def add_rate_of_change(df: pd.DataFrame, target_col: str = "global_active_power_kw") -> pd.DataFrame:
    """Add rate-of-change (first difference) features."""
    out = df.copy()

    # Absolute change per minute
    out[f"{target_col}_diff_1m"] = out[target_col].diff()

    # Absolute change per 5 minutes
    out[f"{target_col}_diff_5m"] = out[target_col].diff(5)

    # Percentage change (handle division by zero)
    prev = out[target_col].shift(1)
    out[f"{target_col}_pct_change_1m"] = (
        out[target_col].diff() / prev.replace(0, np.nan)
    )

    return out

df = add_rate_of_change(df)
diff_cols = [c for c in df.columns if "_diff_" in c or "_pct_change_" in c]
print(f"Rate-of-change features: {diff_cols}")

In [ ]:
# Distribution of 1-minute changes — should be centered near 0 with fat tails
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(df["global_active_power_kw_diff_1m"].dropna(), bins=200,
             color="steelblue", alpha=0.7, edgecolor="white")
axes[0].set_xlabel("Change in Power (kW/min)")
axes[0].set_ylabel("Count")
axes[0].set_title("Distribution of 1-Minute Power Changes")
axes[0].set_xlim(-5, 5)  # Zoom to main distribution

# Q-Q style: sorted absolute changes — tail behaviour
abs_changes = df["global_active_power_kw_diff_1m"].dropna().abs().sort_values()
axes[1].plot(abs_changes.values, linewidth=0.5, color="steelblue")
axes[1].axhline(y=abs_changes.quantile(0.99), color="red", linestyle="--",
                label=f"99th pctl: {abs_changes.quantile(0.99):.2f} kW")
axes[1].set_xlabel("Rank")
axes[1].set_ylabel("|Change| (kW/min)")
axes[1].set_title("Sorted Absolute Changes — Tail Shows Anomalous Jumps")
axes[1].legend()

plt.tight_layout()
plt.show()

## 6. Sub-Metering Ratios

The proportion of energy captured by each sub-meter tells us about
the type of consumption happening at any given time.

In [ ]:
def add_submetering_ratios(df: pd.DataFrame) -> pd.DataFrame:
    """Add sub-metering ratio features."""
    out = df.copy()

    # Total sub-metered (Wh/min)
    out["sub_total_wh"] = (
        out["sub_metering_1_wh"] + out["sub_metering_2_wh"] + out["sub_metering_3_wh"]
    )

    # Total consumption in Wh/min (from kW)
    out["total_wh_per_min"] = out["global_active_power_kw"] * 1000 / 60

    # Ratio of metered to total (0-1, can exceed 1 due to measurement noise)
    out["metered_ratio"] = (
        out["sub_total_wh"] / out["total_wh_per_min"].replace(0, np.nan)
    ).clip(0, 1.5)

    # Individual sub-meter shares
    for i in [1, 2, 3]:
        out[f"sub_{i}_share"] = (
            out[f"sub_metering_{i}_wh"] / out["sub_total_wh"].replace(0, np.nan)
        ).fillna(0)

    return out

df = add_submetering_ratios(df)
ratio_cols = ["metered_ratio", "sub_1_share", "sub_2_share", "sub_3_share"]
print("Sub-metering ratio statistics:")
df[ratio_cols].describe()

## 7. Complete Feature Set Summary

Let's review every feature we've created and check for issues.

In [ ]:
# Full feature list
original_cols = [
    "global_active_power_kw", "global_reactive_power_kw", "voltage_v",
    "global_intensity_a", "sub_metering_1_wh", "sub_metering_2_wh", "sub_metering_3_wh"
]
engineered_cols = [c for c in df.columns if c not in original_cols]

print(f"Original features:    {len(original_cols)}")
print(f"Engineered features:  {len(engineered_cols)}")
print(f"Total features:       {len(df.columns)}")
print(f"\nRows before dropping NaN: {len(df):,}")

# Count NaNs per feature (from lags and rolling windows)
nan_counts = df[engineered_cols].isnull().sum()
nan_pct = (nan_counts / len(df) * 100).round(2)
nan_summary = pd.DataFrame({"nulls": nan_counts, "pct": nan_pct})
print(f"\nFeatures with NaN (from window warmup):")
print(nan_summary[nan_summary["nulls"] > 0].sort_values("nulls", ascending=False))

In [ ]:
# Drop rows with NaN (caused by lag/rolling window warmup)
df_clean = df.dropna()
print(f"Rows after dropping NaN: {len(df_clean):,}")
print(f"Rows dropped: {len(df) - len(df_clean):,} ({100*(len(df)-len(df_clean))/len(df):.2f}%)")
print(f"\nFinal feature matrix shape: {df_clean.shape}")

## 8. Feature Correlation Check

We need to verify that our engineered features actually carry useful
information and aren't perfectly redundant.

In [ ]:
# Correlation of key engineered features with target
target = "global_active_power_kw"
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()
correlations = df_clean[numeric_cols].corrwith(df_clean[target]).abs().sort_values(ascending=False)

print("Top 20 features by absolute correlation with active power:\n")
print(correlations.head(20).to_frame("abs_correlation"))

In [ ]:
# Check for highly correlated feature pairs (potential redundancy)
corr_matrix = df_clean[numeric_cols].corr().abs()

# Get upper triangle pairs with correlation > 0.95
high_corr_pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i + 1, len(corr_matrix.columns)):
        if corr_matrix.iloc[i, j] > 0.95:
            high_corr_pairs.append({
                "feature_1": corr_matrix.columns[i],
                "feature_2": corr_matrix.columns[j],
                "correlation": corr_matrix.iloc[i, j],
            })

if high_corr_pairs:
    pairs_df = pd.DataFrame(high_corr_pairs).sort_values("correlation", ascending=False)
    print(f"Highly correlated pairs (>0.95) — consider dropping one of each:\n")
    print(pairs_df.to_string(index=False))
else:
    print("No feature pairs with correlation > 0.95")

## 9. Summary & Next Steps

### Feature Categories Created

| Category | Count | Examples |
|----------|-------|---------|
| Temporal | ~12 | hour, day_of_week, is_weekend, hour_sin/cos, time_of_use |
| Lag | 4 | 1m, 5m, 15m, 60m lags of active power |
| Rolling | 12 | mean/std/min/max over 1h, 6h, 24h windows |
| Rate of change | 3 | 1m diff, 5m diff, % change |
| Sub-metering | 4 | metered_ratio, sub_1/2/3 share |
| **Total engineered** | **~35** | |

### Key Observations
*Fill in after running:*
1. Most correlated engineered feature with active power: ___
2. Features to consider dropping (redundancy): ___
3. NaN rows dropped due to window warmup: ___

### Next Steps
- Extract all feature functions into `src/features.py`
- Write unit tests in `tests/test_features.py`
- Use this feature matrix for anomaly detection in Notebook 03

In [ ]:
print("Feature engineering notebook complete.")
print(f"Final shape: {df_clean.shape}")